In [87]:
import os, sys, glob

from utils.plotting.standard_modules import *
from utils.particle_tools import nu_tex
#from utils.plotting.percentiles import y_percentiles
from utils.plotting.public_plots import PUBLIC_PLOT_IN_PROGRESS_LABEL_1_LINE, PUBLIC_PLOT_IN_PROGRESS_LABEL_2_LINE

from analysis.oscNext_flercnn.analysis.oscNext_std_osc_analysis import configure_analysis, NEUTRINO_PIPELINE
from analysis.oscNext_flercnn.plotters.common import *
from analysis.oscNext_flercnn.plotters.utils.percentiles import y_percentiles, plot_percentiles


In [89]:
NEUTRINO_PIPELINE='./pipeline_oscNext_neutrinos_std_osc_nh_nocut.cfg'

In [3]:
VERIFICATION_NEUTRINO_PIPELINE='~/Analysis/wg-oscillations-fridge/analysis/oscnext_verification_sample/settings/pipeline/pipeline_oscNext_neutrinos.cfg'

In [4]:
import matplotlib.patches as mpatches

In [5]:
PUBLIC_PLOTS = True

PERCENTILES = [ 50., 16., 84. ]
PERCENTILE_LINESTYLES = [ "-", "--", "--" ]
PERCENTILE_LABELS = [ "Median", r"$1\sigma$", None ]

TRUE_ENERGY_LABEL = r"E_{\rm{true}}"
RECO_ENERGY_LABEL = r"E_{\rm{reco}}"
ENERGY_UNIT = r"[GeV]"
AZIMUTH_UNIT = r"[rad]"
POSITION_UNIT = r"[m]"
LENGTH_UNIT = r"[m]"
TIME_UNIT = r"[ns]"

In [134]:
# setup 
fit_dir=None #where is the real data bfp
output_dir='./flercnn_paper_plots/' #where to save plots

# Choose variables of interest
variables = [
    "weights",
    "true_energy",
    "reco_energy",
    "true_coszen",
    "reco_coszen",
#    "reco_z",
    "pid",
]
cnn_variables = [
    "weights",
    "true_energy",
    "reco_energy",
    "true_coszen",
    "reco_coszen",
    "reco_z",

    "reco_rho",

    "l7_muon_classifier_prob_nu",
    "pid",
]

log=False

In [108]:
veri_physics_params, veri_template_settings, _, veri_template_patches, _ = configure_analysis(
    analysis_name="numu_disappearance", #TODO configurable
    template_settings=VERIFICATION_NEUTRINO_PIPELINE,
)

veri_template_maker, veri_analysis_binning, veri_arrays = get_events(veri_template_settings, param_patches=veri_template_patches, variables=variables)


[    INFO] Loading MCEq spline tables from : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/../..//analysis/common/data/flux/v01/MCEq_flux_gradient_splines_2212_GlobalSplineFitBeta_SIBYLL23C_reduced_scheme.pckl.bz2
[    INFO] Calculating nominal nue flux for nu
[    INFO] Calculating nominal numu flux for nu
[    INFO] Calculating nominal nuebar flux for nu
[    INFO] Calculating nominal numubar flux for nu


In [100]:
#from pisa.core.distribution_maker import DistributionMaker

#template_maker = DistributionMaker([NEUTRINO_PIPELINE])
#veri_template_maker = DistributionMaker([VERIFICATION_NEUTRINO_PIPELINE])

[    INFO] Loading MCEq spline tables from : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/../..//analysis/common/data/flux/v02/MCEq_flux_gradient_splines_2212_GlobalSplineFitBeta_SIBYLL23C.pckl.bz2
[    INFO] Loading MCEq spline tables from : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/../..//analysis/common/data/flux/v01/MCEq_flux_gradient_splines_2212_GlobalSplineFitBeta_SIBYLL23C_reduced_scheme.pckl.bz2


In [101]:
'''template_maker.run()
veri_template_maker.run()
templates = []
veri_templates = []
for p in template_maker.pipelines :
    #p.data.representation = "events"
    p_data = p.data
    templates.append(p_data)

    
for p in veri_template_maker.pipelines :
    #p.data.representation = "events"
    p_data = p.data
    veri_templates.append(p_data)'''

[    INFO] Calculating nominal nue flux for nu
[    INFO] Calculating nominal numu flux for nu
[    INFO] Calculating nominal nuebar flux for nu
[    INFO] Calculating nominal numubar flux for nu
[    INFO] Calculating nominal nue flux for nu
[    INFO] Calculating nominal numu flux for nu
[    INFO] Calculating nominal nuebar flux for nu
[    INFO] Calculating nominal numubar flux for nu


In [102]:
'''from pisa.core.container import ContainerSet
out_cs = ContainerSet("template", containers=templates[0])
veri_out_cs = ContainerSet("template", containers=veri_templates[0])'''

In [119]:
template_patches

{'theta23': {'value': 45.3637 <Unit('degree')>},
 'deltam31': {'value': 0.00247996 <Unit('electron_volt ** 2')>}}

In [135]:
physics_params, template_settings, _, template_patches, _ = configure_analysis(
    analysis_name="numu_disappearance", #TODO configurable
    template_settings=NEUTRINO_PIPELINE,
)

# Get the template (as events)
template_maker, analysis_binning, arrays = get_events(template_settings, param_patches=template_patches, variables=cnn_variables)


[    INFO] Loading MCEq spline tables from : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/../..//analysis/common/data/flux/v02/MCEq_flux_gradient_splines_2212_GlobalSplineFitBeta_SIBYLL23C.pckl.bz2
[    INFO] Calculating nominal nue flux for nu
[    INFO] Calculating nominal numu flux for nu
[    INFO] Calculating nominal nuebar flux for nu
[    INFO] Calculating nominal numubar flux for nu


In [129]:
#
# Plot reconstruction performance
# 

# Choose binning
if log :
    energy_bins = np.logspace(np.log10(5.), np.log10(10000.), num=20)
    # energy_bins = np.logspace(np.log10(analysis_binning["pid"].bin_edges.m[0]), np.log10(analysis_binning["pid"].bin_edges.m[-1]), num=21)
    energy_scale = "log"
else :
    energy_bins = get_bins(5., 100., num=24)
    energy_scale = "linear"
    coszen_bins = get_bins(-1., 0., num=24)
    z_bins = get_bins(-1500., -2000., num=50)

In [31]:
import matplotlib as mpl
TITLE_FONT = 20
def pretty():
    mpl.rcParams['xtick.labelsize'] = TITLE_FONT 
    mpl.rcParams['ytick.labelsize'] = TITLE_FONT 
    mpl.rcParams['axes.labelsize'] = TITLE_FONT 
    mpl.rcParams['axes.titlesize'] = TITLE_FONT 

In [56]:
def plot_reco_error_vs_energy(
    arrays,
    energy_bins,
    energy_scale,
    error_bins,
    true_key,
    reco_key,
    true_key2,
    reco_key2,
    true_label,
    reco_label,
    true_label2,
    reco_label2,
    unit=None,
    rad2deg=False,
    output_dir=None,
    include_events_hist=True,
) :
    '''
    Plot the reco resolution for the parameter of interest

    The error definitiion is: 
        fractional_error == False : reco - true
        fractional_error == True  : ( reco - true ) / true
    '''

    #TODO Also add 1D plots (e.g. 1D hists for each energy bin)

    if unit is None :
        unit = ""
    else :
        unit = r" " + unit


    #
    # Loop over data categories
    #

    for particle_key, particle_arrays in arrays.items() :

        if particle_key != "numu_cc" :
            continue  
        # Create the figure
        fig = Figure(figsize=(7, 5))


        #
        # Get parameters
        #

        # Extract inputs
        true_energy = particle_arrays["true_energy"]
        true_var = particle_arrays[true_key]
        reco_var = particle_arrays[reco_key]
        weights = particle_arrays["weights"]
        true_var2 = particle_arrays[true_key2]
        reco_var2 = particle_arrays[reco_key2]
        
        # Unit conversions
        if rad2deg :
            true_var = np.rad2deg(true_var)
            reco_var = np.rad2deg(reco_var)
            unit = " [deg]"

        # Calc error

        reco_error = reco_var - true_var
        reco_error2 = ( reco_var2 - true_var2 ) / true_var2 # In %
        
        med = np.median(reco_error)
        med2 = np.median(reco_error2)

        #
        # Plot true energy vs variable reco error
        #

        # Get ax
        ax = fig.get_ax()
        ax2 = ax.twinx()  
        
        # Fill hist and plot it
        if include_events_hist :
            hist = Histogram( 2, uof=True, bins=[energy_bins, error_bins], x=true_energy, y=reco_error, weights=weights )
            cmesh = plot_hist( ax=ax, hist=hist, text=False, edges=False, cmap="Oranges" )
            addColorbar( fig=fig, ax=ax, cmesh=cmesh, label="Num events" )

        # Plot the y percentiles
        xmin = energy_bins[0]
        xmax = energy_bins[-1]
        xwidth = ( xmax - xmin ) / 50.
        nx = 25
        pc_x, pc_y, pc_masks = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_energy,
            y=reco_error,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )
        pc_x2, pc_y2, pc_masks2 = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_energy,
            y=reco_error2,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )

        # Plot the y percentiles
        if include_events_hist :
            y_percentiles(
                ax=ax,
                x=pc_x,
                y_percentiles=pc_y,
                y_percentile_masks=pc_masks,
                binned=False,
                percentiles=PERCENTILES,
                percentile_linestyles=PERCENTILE_LINESTYLES,
                percentile_labels=PERCENTILE_LABELS,
                color="red",
            )
        else :#PERCENTILE_LABELS[0]
            ax.plot(pc_x, pc_y[0], color="red", label=r"$%s$" % (true_label), zorder=5,alpha=0.5)
            ax.fill_between(pc_x, pc_y[1], pc_y[2], color="orange", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
            ax2.plot(pc_x2, pc_y2[0], color="blue", label="Energy", zorder=5,alpha=0.5)
            ax2.fill_between(pc_x2, pc_y2[1], pc_y2[2], color="blue", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
        patch = mpatches.Patch(color='grey', label=r'1$\sigma$')

        # Fix scale
        ax.set_ylim(-1, 1)
        
        ax2.set_ylim(-1.0, 1.0)
        hands, labels = ax.get_legend_handles_labels()
        hands2, labels2 = ax2.get_legend_handles_labels()

        # Add best case scenario line
        ax.axhline( y=0, color="black", zorder=2, linestyle='--' )
#        ax.axhline( y = 0.0625-0.02572333, color="red", zorder=4 )
        # Labels
        ax2.set_ylabel( r"(Reco. - True) / True $E_{\nu}$")
        ax.set_ylabel(r"Reco. - True $%s$" % (true_label))

        # Public plot label
        if False:
            ax.text(0.99, 0.01, PUBLIC_PLOT_IN_PROGRESS_LABEL_1_LINE, horizontalalignment='right', verticalalignment='bottom',transform=ax.transAxes, fontsize=12, color="red")

        # Format
        fig.quick_format( xlabel=r"True Neutrino Energy %s"%(ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )

        #        fig.quick_format( xlabel=r"$%s$ %s"%(TRUE_ENERGY_LABEL,ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )
        ax.legend(handles=[hands[0], hands2[0], patch], loc="lower left")
        if output_dir is not None :
            fig.save( os.path.join( output_dir, "%s_%s_reco_error_%s.png"%(reco_key.replace("reco_",""), particle_key, energy_scale) ) )


In [91]:
def plot_reco_vs_true(
    arrays1,
    arrays2,
    leg1,
    leg2,
    bins,
    scale,
    true_key,
    reco_key,
    true_label,
    reco_label,
    unit=None,
    rad2deg=False,
    output_dir=None,
    include_events_hist=True,
) :
    '''
    Plot the reco resolution for the parameter of interest

    The error definitiion is: 
        fractional_error == False : reco - true
        fractional_error == True  : ( reco - true ) / true
    '''

    #TODO Also add 1D plots (e.g. 1D hists for each energy bin)

    if unit is None :
        unit = ""
    else :
        unit = r" " + unit


    #
    # Loop over data categories
    #

    for particle_key, particle_arrays in arrays1.items() :

        if particle_key != "numu_cc" :
            continue  
        # Create the figure
        fig = Figure(figsize=(8, 6))


        #
        # Get parameters
        #

        # Extract inputs
        true_var1 = particle_arrays[true_key]
        reco_var1 = particle_arrays[reco_key]
        weights1 = particle_arrays["weights"]
        
        for particle_key2, particle_arrays2 in arrays2.items() :

            if particle_key2 != "numu_cc" :
                continue  

            # Get parameters
            #

            # Extract inputs
            true_var2 = particle_arrays2[true_key]
            reco_var2 = particle_arrays2[reco_key]
            weights2 = particle_arrays2["weights"]


        
        # Unit conversions
        if rad2deg :
            true_var2 = np.rad2deg(true_var2)
            reco_var2 = np.rad2deg(reco_var2)
            unit = " [deg]"

        #
        # Plot true energy vs variable reco error
        #

        # Get ax
        ax = fig.get_ax()
        #ax2 = ax.twinx()  
        
        # Fill hist and plot it
        if include_events_hist :
            hist = Histogram( 2, uof=True, bins=[bins, bins], x=true_var1, y=reco_var1, weights=weights1)
            cmesh = plot_hist( ax=ax, hist=hist, text=False, edges=False, cmap="Oranges" )
            addColorbar( fig=fig, ax=ax, cmesh=cmesh, label="Num events" )

        # Plot the y percentiles
        xmin = bins[0]
        xmax = bins[-1]
        xwidth = ( xmax - xmin ) / 10.
        nx = 10
        pc_x, pc_y, pc_masks = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_var1,
            y=reco_var1,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )
        print(pc_y)
        pc_x2, pc_y2, pc_masks2 = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_var2,
            y=reco_var2,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )

        # Plot the y percentiles
        if include_events_hist :
            plot_percentiles(
                ax=ax,
                x=pc_x,
                y_percentiles=pc_y,
                y_percentile_masks=pc_masks,
                binned=False,
                percentiles=PERCENTILES,
                percentile_linestyles=PERCENTILE_LINESTYLES,
                percentile_labels=PERCENTILE_LABELS,
                color="red",
            )
        else :#PERCENTILE_LABELS[0]
            ax.plot(pc_x, pc_y[0], color="red", label=leg1, zorder=5,alpha=0.5)
            ax.fill_between(pc_x, pc_y[1], pc_y[2], color="orange", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
            ax.plot(pc_x2, pc_y2[0], color="blue", label=leg2, zorder=5,alpha=0.5)
            ax.fill_between(pc_x2, pc_y2[1], pc_y2[2], color="blue", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)
        patch = mpatches.Patch(color='grey', label=r'1$\sigma$')

        # Fix scale
        #ax.set_ylim(-1, 1)
        
        #ax2.set_ylim(-1.0, 1.0)
        hands, labels = ax.get_legend_handles_labels()
        #hands2, labels2 = ax2.get_legend_handles_labels()

        # Add best case scenario line
        #ax.axhline( y=0, color="black", zorder=2, linestyle='--' )
#        ax.axhline( y = 0.0625-0.02572333, color="red", zorder=4 )
        # Labels
        #ax.set_ylabel( r"(Reco. - True) / True $E_{\nu}$")
        ax.set_ylabel(reco_label)
        #r"Reco. - True $%s$" % (true_label))

        # Public plot label
        if False:
            ax.text(0.99, 0.01, PUBLIC_PLOT_IN_PROGRESS_LABEL_1_LINE, horizontalalignment='right', verticalalignment='bottom',transform=ax.transAxes, fontsize=12, color="red")

        # Format
        fig.quick_format( xlabel=true_label, xscale=energy_scale, xlim=(xmin, xmax) )

        #        fig.quick_format( xlabel=r"$%s$ %s"%(TRUE_ENERGY_LABEL,ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )
        ax.legend()#handles=[hands[0], hands2[0], patch], loc="lower left")
        if output_dir is not None :
            fig.save( os.path.join( output_dir, "%s_%s_reco_vs_true_%s.png"%(reco_key.replace("reco_",""), particle_key, energy_scale) ) )


In [121]:
plot_reco_vs_true(
    arrays1=arrays,
    arrays2=veri_arrays,
    leg1 = "CNN",
    leg2 = "SANTA",
    bins=energy_bins,
    scale=energy_scale,
    #error_bins=get_bins(-2., 2., num=20),
    true_key='true_energy',
    reco_key="reco_energy",
    true_label="True neutrino energy [GeV]", #r"\cos(\theta_{\rm{zenith}})",
    reco_label="Reco neutrino energy [GeV]", #r"\cos(\theta_{\rm{zenith}})",
    unit=ENERGY_UNIT,
    rad2deg=False,
    output_dir=output_dir,
    include_events_hist=False,
)

[array([ 7.92551339, 14.89636302, 25.78869164, 37.26192564, 48.31542969,
       58.5901916 , 67.60833859, 74.22122359, 78.78795862, 81.91896081]), array([ 5.77711651,  9.22151893, 16.85493654, 25.22203755, 33.61905313,
       41.65365529, 49.07040238, 55.04924583, 59.46559429, 61.67157578]), array([11.88598099, 22.30926037, 36.92557263, 51.57948017, 65.40243125,
       76.95295191, 85.41810393, 90.28607821, 92.97416377, 94.58859897])]
Saved image : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/plotters/publication/flercnn_paper_plots/energy_numu_cc_reco_vs_true_linear.png


In [123]:
plot_reco_vs_true(
    arrays1=arrays,
    arrays2=veri_arrays,
    leg1 = "CNN",
    leg2 = "SANTA",
    bins=coszen_bins,
    scale=energy_scale,
    #error_bins=get_bins(-2., 2., num=20),
    true_key='true_coszen',
    reco_key="reco_coszen",
    true_label="True cos(zen)", #r"\cos(\theta_{\rm{zenith}})",
    reco_label="Reco cos(zen)", #r"\cos(\theta_{\rm{zenith}})",
    unit=ENERGY_UNIT,
    rad2deg=False,
    output_dir=output_dir,
    include_events_hist=False,
)

[array([-0.92153349, -0.85452386, -0.75712956, -0.65436628, -0.55083165,
       -0.45141052, -0.35467451, -0.25659002, -0.16413022, -0.07792675]), array([-0.97914464, -0.94290514, -0.89338126, -0.83941308, -0.77600971,
       -0.70481786, -0.62286991, -0.54279444, -0.46113282, -0.38740905]), array([-0.71618144, -0.61379643, -0.48957159, -0.3785037 , -0.27756653,
       -0.18742704, -0.10190641, -0.0230034 ,  0.05153525,  0.12110109])]
Saved image : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/plotters/publication/flercnn_paper_plots/coszen_numu_cc_reco_vs_true_linear.png


In [130]:
def plot_reco_vs_true_single(
    arrays1,
    leg1,
    bins,
    scale,
    true_key,
    reco_key,
    true_label,
    reco_label,
    unit=None,
    rad2deg=False,
    output_dir=None,
    include_events_hist=True,
) :
    '''
    Plot the reco resolution for the parameter of interest

    The error definitiion is: 
        fractional_error == False : reco - true
        fractional_error == True  : ( reco - true ) / true
    '''

    #TODO Also add 1D plots (e.g. 1D hists for each energy bin)

    if unit is None :
        unit = ""
    else :
        unit = r" " + unit


    #
    # Loop over data categories
    #

    for particle_key, particle_arrays in arrays1.items() :

        if particle_key != "numu_cc" :
            continue  
        # Create the figure
        fig = Figure(figsize=(8, 6))


        #
        # Get parameters
        #

        # Extract inputs
        true_var1 = particle_arrays[true_key]
        reco_var1 = particle_arrays[reco_key]
        weights1 = particle_arrays["weights"]
    
        #
        # Plot true energy vs variable reco error
        #

        # Get ax
        ax = fig.get_ax()
        #ax2 = ax.twinx()  
        
        # Fill hist and plot it
        if include_events_hist :
            hist = Histogram( 2, uof=True, bins=[bins, bins], x=true_var1, y=reco_var1, weights=weights1)
            cmesh = plot_hist( ax=ax, hist=hist, text=False, edges=False, cmap="Oranges" )
            addColorbar( fig=fig, ax=ax, cmesh=cmesh, label="Num events" )

        # Plot the y percentiles
        xmin = bins[0]
        xmax = bins[-1]
        xwidth = ( xmax - xmin ) / 10.
        nx = 10
        pc_x, pc_y, pc_masks = y_percentiles(
            xmin=xmin,
            xmax=xmax,
            xwidth=xwidth,
            nx=nx,
            x=true_var1,
            y=reco_var1,
            percentiles=PERCENTILES,
            percentile_linestyles=PERCENTILE_LINESTYLES,
            percentile_labels=PERCENTILE_LABELS,
        )

        # Plot the y percentiles
        if include_events_hist :
            plot_percentiles(
                ax=ax,
                x=pc_x,
                y_percentiles=pc_y,
                y_percentile_masks=pc_masks,
                binned=False,
                percentiles=PERCENTILES,
                percentile_linestyles=PERCENTILE_LINESTYLES,
                percentile_labels=PERCENTILE_LABELS,
                color="red",
            )
        else :#PERCENTILE_LABELS[0]
            ax.plot(pc_x, pc_y[0], color="red", label=leg1, zorder=5,alpha=0.5)
            ax.fill_between(pc_x, pc_y[1], pc_y[2], color="orange", label=PERCENTILE_LABELS[1], zorder=3, alpha=0.5)

        # Fix scale
        #ax.set_ylim(-1, 1)
        
        #ax2.set_ylim(-1.0, 1.0)
        hands, labels = ax.get_legend_handles_labels()
        #hands2, labels2 = ax2.get_legend_handles_labels()

        # Add best case scenario line
        #ax.axhline( y=0, color="black", zorder=2, linestyle='--' )
#        ax.axhline( y = 0.0625-0.02572333, color="red", zorder=4 )
        # Labels
        #ax.set_ylabel( r"(Reco. - True) / True $E_{\nu}$")
        ax.set_ylabel(reco_label)
        #r"Reco. - True $%s$" % (true_label))

        # Public plot label
        if False:
            ax.text(0.99, 0.01, PUBLIC_PLOT_IN_PROGRESS_LABEL_1_LINE, horizontalalignment='right', verticalalignment='bottom',transform=ax.transAxes, fontsize=12, color="red")

        # Format
        fig.quick_format( xlabel=true_label, xscale=energy_scale, xlim=(xmin, xmax) )

        #        fig.quick_format( xlabel=r"$%s$ %s"%(TRUE_ENERGY_LABEL,ENERGY_UNIT), xscale=energy_scale, xlim=(xmin, xmax) )
        ax.legend()#handles=[hands[0], hands2[0], patch], loc="lower left")
        if output_dir is not None :
            fig.save( os.path.join( output_dir, "%s_%s_reco_vs_true_%s_single.png"%(reco_key.replace("reco_",""), particle_key, energy_scale) ) )


In [141]:
plot_reco_error_vs_energy(
    arrays=arrays,
    energy_bins=energy_bins,
    energy_scale=energy_scale,
    error_bins=get_bins(-2., 2., num=20),
    true_key='true_coszen',
    reco_key="reco_coszen",
    true_key2="true_energy",
    reco_key2="reco_energy",
    true_label=r"\cos(\theta_{\rm{zenith}})",
    reco_label=r"\cos(\theta_{\rm{zenith}})",
    true_label2=r"True E_{\nu}", #E_{\rm{true}}",
    reco_label2=r"E_{\rm{reco}}",
    unit=ENERGY_UNIT,
    output_dir=output_dir,
    include_events_hist=False,
)

Saved image : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/plotters/publication/flercnn_paper_plots/coszen_numu_cc_reco_error_linear.png


In [140]:
plot_reco_error_vs_energy(
    arrays=veri_arrays,
    energy_bins=energy_bins,
    energy_scale=energy_scale,
    error_bins=get_bins(-2., 2., num=20),
    true_key='true_coszen',
    reco_key="reco_coszen",
    true_key2="true_energy",
    reco_key2="reco_energy",
    true_label=r"\cos(\theta_{\rm{zenith}})",
    reco_label=r"\cos(\theta_{\rm{zenith}})",
    true_label2=r"True E_{\nu}", #E_{\rm{true}}",
    reco_label2=r"E_{\rm{reco}}",
    unit=ENERGY_UNIT,
    output_dir=output_dir,
    include_events_hist=False,
)

Saved image : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/plotters/publication/flercnn_paper_plots/coszen_numu_cc_reco_error_linear.png


Saved image : /mnt/ufs18/home-033/yushiqi2/Analysis/wg-oscillations-fridge/analysis/oscNext_flercnn/plotters/publication/flercnn_paper_plots/coszen_numu_cc_reco_vs_true_log.png
